In [1]:
import pandas as pd

csv_path = "../WebScraping/Transactions.csv"

df = pd.read_csv(csv_path)
print(df.head())



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_lo

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "C:\Users\datta\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_lo

AttributeError: _ARRAY_API not found

            Txn Hash           Type   Block               From  \
0  0x7bf7d...e5f4b87  Coin_Transfer  485665  0x4E5FF...9CC3C31   
1  0x4774f...a4fe408  Coin_Transfer  476179  0x400D4...d27cE1C   
2  0xd6147...7613120  Coin_Transfer  469467  0x400D4...d27cE1C   
3  0x08830...cbffce7  Coin_Transfer  463260  0x400D4...d27cE1C   
4  0x75689...541cc33  Coin_Transfer  451243  0x400D4...d27cE1C   

                  To                       Timestamp   Txn Fee  Value(CINT)  \
0  0x1f434...88444e0  2d ago\r\nJul, 15, 2025, 03:03  0.000008          0.1   
1  0xA48E1...a0D9630  2d ago\r\nJul, 14, 2025, 11:11  0.000000         20.0   
2  0x67694...b304ccf  3d ago\r\nJul, 13, 2025, 23:57  0.000000         20.0   
3  0x6bf85...1Aec29a  3d ago\r\nJul, 13, 2025, 13:34  0.000000         20.0   
4  0x75619...079EAB9  4d ago\r\nJul, 12, 2025, 17:27  0.000000         20.0   

    Status  
0  Success  
1  Success  
2  Success  
3  Success  
4  Success  


In [2]:
print(df.columns.tolist())


['Txn Hash', 'Type', 'Block', 'From', 'To', 'Timestamp', 'Txn Fee', 'Value(CINT)', 'Status']


In [4]:
import pandas as pd
import numpy as np

# 1. Load CSV
csv_path = "../WebScraping/Transactions.csv"
df = pd.read_csv(csv_path)

# 2. Drop duplicate Txn Hashes
#df = df.drop_duplicates(subset=["Txn Hash"])

# 3. Standardize & rename columns
df = df.rename(columns=lambda c: (
    c.strip()
     .lower()
     .replace(" ", "_")
     .replace("(", "")
     .replace(")", "")
     .replace("-", "_")
))

df = df.rename(columns={
    "valuecint": "value_cint",    # rename the raw
    "txn_fee":   "txn_fee_cint",
})
# now drop any leftover
if "valuecint" in df.columns:
    df = df.drop(columns=["valuecint"])
# e.g. "Txn Hash" → "txn_hash", "Txn Fee (CINT)" → "txn_fee_cint"

# 4. Parse and normalize timestamps
df["timestamp_clean"] = (
    df["timestamp"]
      .astype(str)
      .str.split("\n")
      .str[-1]                # take the 'Jul, 15, 2025, 03:03' portion
)

df["timestamp"] = pd.to_datetime(df["timestamp_clean"], format="%b, %d, %Y, %H:%M", errors="coerce")
df = df.drop(columns="timestamp_clean")

# 5. Normalize address fields
for col in ["from", "to", "contract_address"]:
    if col in df:
        df[col] = df[col].str.lower().str.strip().replace("", np.nan)

# 6. Convert all on-chain numeric columns to floats/ints
df["block"]        = df["block"].astype(int)
df["value_cint"]   = df["value_cint"].astype(float)
df["txn_fee_cint"] = df["txn_fee_cint"].astype(float)


# 7. Derive domain-specific fields

# 7b. Classify txn direction for a given “our” address set
#    (you can supply your own key-addresses list)
our_addrs = {"0xabc…", "0xdef…"}  
def direction(row):
    if row["from"] in our_addrs and row["to"] not in our_addrs:
        return "outgoing"
    elif row["to"] in our_addrs and row["from"] not in our_addrs:
        return "incoming"
    else:
        return "other"
df["direction"] = df.apply(direction, axis=1)

# 8. Drop or archive any truly irrelevant fields
#    e.g., if ‘status’ is always ‘Success’, drop it; if you’ll never need raw input_data, drop that.
if "status" in df and df["status"].nunique() == 1:
    df = df.drop(columns=["status"])
for col in ["input_data", "logs"] :
    if col in df:
        df = df.drop(columns=[col])

# 9. Sort & re-index
df = df.sort_values("timestamp").reset_index(drop=True)

# 10. Final integrity checks
#assert df["txn_hash"].is_unique, "Txn Hash is not unique!"
assert df[["block", "timestamp"]].notnull().all().all(), "Missing block or timestamp!"



df = df.rename(columns={
    "from": "from_address",
    "to":   "to_address"
})

df.head(10)

output_path = "Transactions_cleaned.csv"
df.to_csv(output_path, index=False)
